In [4]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import random
import string
import math
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

In [5]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [3]:
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before

allRatings = []
userRatings = defaultdict(list)

for user,recipe,d in readCSV("trainInteractions.csv.gz"):
  r = int(d['rating'])
  allRatings.append(r)
  userRatings[user].append(r)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

predictions = open("predictions_Rated.txt", 'w')
for l in open("stub_Rated.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if u in userAverage:
    predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
  else:
    predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

predictions.close()

In [28]:
### Would-cook baseline: just rank which recipes are popular and which are not, and return '1' if a recipe is among the top-ranked

recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV("trainInteractions.csv.gz"):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

# return1 = set()
# count = 0
# for ic, i in mostPopular:
#   count += ic
#   return1.add(i)
#   if count > 0.75 * totalCooked: break

# predictions = open("predictions_Made.txt", 'w')
# for l in open("stub_Made.txt"):
#   if l.startswith("user_id"):
#     #header
#     predictions.write(l)
#     continue
#   u,i = l.strip().split('-')
#   if i in return1:
#     predictions.write(u + '-' + i + ",1\n")
#   else:
#     predictions.write(u + '-' + i + ",0\n")

# predictions.close()

In [6]:
### Cook-time prediction baseline: Regress based on the length of the instructions

X = []
y = []

for d in readGz("trainRecipes.json.gz"):
  X.append([1,len(d['steps'])])
  y.append(d['minutes'])

mod = linear_model.LinearRegression()
mod.fit(X, y)

predictions = open("predictions_Minutes.txt", 'w')
predictions.write("recipe_id,prediction\n")
for d in readGz("testRecipes.json.gz"):
  x = [1,len(d['steps'])]
  pred = mod.predict([x])[0]
  predictions.write(d['recipe_id'] + ',' + str(pred) + '\n')

predictions.close()

In [6]:
dataset = [(user,recipe) for user,recipe,d in readCSV("trainInteractions.csv.gz")]
Train = dataset[:150000]
Valid = dataset[150000:]

itemsPerUser = defaultdict(set)
usersPerItem = defaultdict(set)

for user, recipe in dataset:
    itemsPerUser[user].add(recipe)
    usersPerItem[recipe].add(user)

allRecipes = []
for d in readGz("trainRecipes.json.gz"):
    allRecipes.append(d['recipe_id'])

extra = []
for user,recipe in Valid:
    rand = random.choice(allRecipes)
    while rand in itemsPerUser[user]:
        rand = random.choice(allRecipes)
    extra.append((user, rand))
Valid = Valid + extra

In [7]:
ValidY = [1]*100000 + [0]*100000

In [8]:
itemsPerUserTrain = defaultdict(set)
usersPerItemTrain = defaultdict(set)

for user, recipe in Train:
    itemsPerUserTrain[user].add(recipe)
    usersPerItemTrain[recipe].add(user)

In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2)) 
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [10]:
#predictions = [predict(user, recipe) for (user, recipe) in Valid]

recipeCount = defaultdict(int)
totalCooked = 0

for user,recipe,_ in readCSV("trainInteractions.csv.gz"):
  recipeCount[recipe] += 1
  totalCooked += 1

mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.75 * totalCooked: break

len(mostPopular)

151462

In [11]:
mostPopular = [(recipeCount[x], x) for x in recipeCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.75 * totalCooked: break

preds = []
for user, recipe in Valid[95000:105000]:
    if len(itemsPerUser[user])>:
        preds.append(1);
    else:
        preds.append(0);
        
accuracy = [not a^b for (a,b) in zip(preds, ValidY[95000:105000])]
print("q2:", sum(accuracy) / len(accuracy))

SyntaxError: invalid syntax (962575869.py, line 14)

In [12]:
data = [d for user,recipe,d in readCSV("trainInteractions.csv.gz")]
data[0]

{'user_id': '88348277',
 'recipe_id': '03969194',
 'date': '2004-12-23',
 'rating': '5'}

In [13]:
rec = [d for d in readGz("trainRecipes.json.gz")]

In [14]:
ingPerRecipe = defaultdict(set)

for d in readGz("trainRecipes.json.gz"):
    for i in d['ingredients']:
        ingPerRecipe[d['recipe_id']].add(i)

In [15]:
mpUser = {}
for u in itemsPerUser:
    mpUser[u] = len(itemsPerUser[u])

#similarities.sort(reverse=True)
#mpUser = sorted(mpUser, key=lambda s: (-s[0]))
mpUser = sorted(mpUser.items(), key=lambda x: x[1], reverse=True)


In [16]:
returnU = set()
count = 0
for u in mpUser:
  count += mpUser[u]
  returnU.add(u)
  if count > 0.735666 * totalCooked: break

TypeError: list indices must be integers or slices, not tuple

In [17]:
list(pct)[151400:]
pct['00068256']

NameError: name 'pct' is not defined

In [18]:
totalCooked

500000

In [19]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.735666 * totalCooked: break

pct = {}
count = 0
for ic, i in mostPopular:
    count += ic
    pct[i] = count / totalCooked
    
def feat(user, recipe):
    similarities = []
    f = [0, 0]
    for r in itemsPerUser[user]:
        if r==recipe: continue
        sim = Jaccard(usersPerItem[recipe], usersPerItem[r])
        similarities.append(sim)
    if len(similarities)==0: return [0,0]
    avg=sum(similarities)/len(similarities)
    m=max(similarities)
    return [avg, m]

def feat3(user, recipe):
    if recipe in return1: return 1
    similarities = []
    for r in itemsPerUser[user]:
        if r==recipe: continue
        sim = Jaccard(usersPerItem[recipe], usersPerItem[r])
        similarities.append(sim)
    if(len(similarities)==0): return 0
    avg=sum(similarities)/len(similarities)
    return avg

def feat1(user, recipe):
    #if recipe in return1: return 1
    maxSim = 0
    maxSim1 = 0
    for r in itemsPerUser[user]:
        if r==recipe: continue
        sim = Jaccard(usersPerItem[recipe], usersPerItem[r])
        if sim > maxSim: maxSim = sim
        sim2 = Jaccard(ingPerRecipe[recipe], ingPerRecipe[r])
        if sim2 > maxSim1: maxSim1 = sim2
        ## new recipe cold start
#         if len(usersPerItem[recipe])==0: 
#             sim2 = Jaccard(ingPerRecipe[recipe], ingPerRecipe[r])
#             if sim2 > maxSim: maxSim = sim2
#         else:
#             sim = Jaccard(usersPerItem[recipe], usersPerItem[r])
#             if sim > maxSim: maxSim = sim
#     return maxSim
    f = 0
    if recipe in pct: f = pct[recipe]
    #return [f, (recipe in return1), maxSim, maxSim1, 1]
    return [f, maxSim, maxSim1, 1]

def feat2(user, recipe):
#     if recipe in pct: return [1, pct[recipe]]
#     else: return [1, 0]
    if recipe in return1: return [1, 1]
    return [1, 0]

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [21]:
negTrain = []
for user,recipe in Train[:10000]:
    rand = random.choice(allRecipes)
    while rand in itemsPerUser[user]:
        rand = random.choice(allRecipes)
    negTrain.append((user, rand))
len(negTrain)

trainSet = Train[:10000] + negTrain
print(len(trainSet))

X = [feat1(user, recipe) for (user, recipe) in trainSet]
Y = [1]*10000 + [0]*10000

print("done")

20000
done


In [22]:
mod = KNeighborsClassifier(n_neighbors=100)
mod.fit(X,Y)
print("done fit")
#mod = linear_model.LogisticRegression(class_weight='balanced')
#mod.fit(X,Y)

xValid = [feat1(user, recipe) for (user, recipe) in Valid[90000:110000]]
pds = mod.predict(xValid)
print("done predict")

accuracy = [not a^b for (a,b) in zip(pds, ValidY[90000:110000])]
print(sum(accuracy)/len(accuracy))

done fit
done predict
0.94315


In [23]:
sum(pds)

10645

In [24]:
xValid = [[1, feat1(user, recipe)] for (user, recipe) in Valid]
pds = mod.predict(xValid)
accuracy = [not a^b for (a,b) in zip(pds, ValidY)]
print(sum(accuracy)/len(accuracy))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'

In [ ]:
len(Train)

In [112]:
negTrain = []
for user,recipe in Train:
    rand = random.choice(allRecipes)
    while rand in itemsPerUser[user]:
        rand = random.choice(allRecipes)
    negTrain.append((user, rand))
len(negTrain)

trainSet = Train + negTrain
print(len(trainSet))

X = [[1, feat1(user, recipe)] for (user, recipe) in trainSet]
Y = [1]*400000 + [0]*400000


#mod = KNeighborsClassifier(n_neighbors=3)
mod = linear_model.LogisticRegression(class_weight='balanced')
mod.fit(X,Y)

xValid = [[1, feat1(user, recipe)] for (user, recipe) in Valid[90000:110000]]
pds = mod.predict(xValid)
accuracy = [not a^b for (a,b) in zip(pds, ValidY[90000:110000])]
print(sum(accuracy)/len(accuracy))

800000
0.8237


In [722]:
mod = linear_model.LogisticRegression(class_weight='balanced')
mod.fit(X,Y)

LogisticRegression(class_weight='balanced')

In [187]:
pds = []

predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  test = feat1(u, i)
  pd = mod.predict([test])[0]
  pds.append(pd)
  predictions.write(u + '-' + i + "," + str(pd) + "\n")

predictions.close()

print("done")
sum(pds), len(pds)

done


(5191, 20000)

In [183]:
sum(pds), len(pds)

(5226, 20000)

In [111]:
len(pds)

20000

In [723]:
X[0]

[1, 1, 1]

In [724]:
# xTest = [[1, feat(user, recipe), feat2(user, recipe)] for (user, recipe) in Train]
# yTest = [1]*400000

pds = mod.predict(X)
accuracy = [not a^b for (a,b) in zip(pds, Y)]
sum(accuracy)/len(accuracy)

0.82

In [728]:
xValid = [[1, feat1(user, recipe), feat2(user, recipe)] for (user, recipe) in Valid[95000:105000]]
pds = mod.predict(xValid)
accuracy = [not a^b for (a,b) in zip(pds, ValidY[95000:105000])]
sum(accuracy)/len(accuracy)

0.8157

In [686]:
pd = [((recipe in return1)|(predict(user, recipe))) for (user, recipe) in Valid]
accuracy = [not a^b for (a,b) in zip(pd, ValidY)]
print("pd:", sum(accuracy) / len(accuracy))

pd: 0.68016


In [ ]:
def predict(user, recipe):
    for r in itemsPerUserTrain[user]:
        if r==recipe: return 1
        sim = Jaccard(ingPerRecipe[recipe], ingPerRecipe[r]) + Jaccard(usersPerItemTrain[recipe], usersPerItemTrain[r])
        if sim > 0.2: return 1
    return 0

In [283]:
pd[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [234]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.73 * totalCooked: break

pd2 = [(recipe in return1) for (user, recipe) in Valid]
accuracy2 = [not a^b for (a,b) in zip(pd2, ValidY)]
print("pd2:", sum(accuracy2) / len(accuracy2))

pd2: 0.745885


In [235]:
pd = [((recipe in return1)|(predict(user, recipe))) for (user, recipe) in Valid]
accuracy = [not a^b for (a,b) in zip(pd, ValidY)]
print("q3:", sum(accuracy) / len(accuracy))

q3: 0.67587


In [219]:
len(Valid)

200000

In [204]:
return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > 0.75 * totalCooked: break

predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
  if l.startswith("user_id"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  if i in return1:
    predictions.write(u + '-' + i + ",1\n")
  else:
    predictions.write(u + '-' + i + "," + str(predict(u,i)) + "\n")

predictions.close()

In [268]:
similarities = []
for r in itemsPerUserTrain[data[1]['user_id']]:
    for q in itemsPerUserTrain[data[1]['user_id']]:
        if r==q: continue
        sim = Jaccard(ingPerRecipe[q], ingPerRecipe[r])
        similarities.append((sim, q, r))
#similarities.sort(reverse=True)
similarities = sorted(similarities, key=lambda s: (-s[0]))

In [267]:
len(itemsPerUserTrain[data[1]['user_id']])

9

In [265]:
ingPerRecipe['90743062']

{'cloves', 'milk', 'sugar', 'tea leaves', 'water'}

In [266]:
ingPerRecipe['69440641']

{'milk', 'sugar', 'tea leaves', 'water'}

In [25]:
data = []

for d in readGz("trainRecipes.json.gz"):
    data.append(d)
    
NTrain = (9*len(data))//10
dataTrain = data[:NTrain]

sp = set(string.punctuation)

wordCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['steps'].lower() if not c in sp])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

NW = 1000

words = [x[1] for x in counts[:NW]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [26]:
df = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['steps'].lower() if not c in sp])
    for w in set(r.split()):
        df[w] += 1

In [28]:
rev = data[9] # Query review
rev

{'name': 'bakesale betty s fried chicken sandwich',
 'minutes': 105,
 'contributor_id': '87853888',
 'submitted': '2008-02-22',
 'steps': 'season chicken breasts with kosher salt\tlet sit at least 5 minutes\tfill a wide , shallow nonreactive bowl or casserole dish with buttermilk\tadd the chicken and soak in the refrigerator for 1 hour up to overnight\tfor the vinaigrette: combine mustard , vinegar and salt in a bowl\tslowly whisk in olive oil until well blended\tfor the coleslaw: macerate onions in the cup of red wine vinegar , and let sit at least 20 minutes\tremove onions and discard vinegar\ttoss onions with jalapeno , parsley , cabbage and salt\ttoss with vinaigrette until evenly coated\tto fry chicken: pour vegetable oil into a large stockpot\tdo not fill up more than halfway , or the oil could splatter\tbring oil up to 365 , using a digital thermometer / candy thermometer to monitor the heat\tprepare the the breading while waiting for oil to heat up\tin a wide shallow bowl , mix

In [31]:
tf = defaultdict(int)
r = ''.join([c for c in rev['steps'].lower() if not c in sp])
for w in r.split():
    # Note = rather than +=, different versions of tf could be used instead
    tf[w] = 1
    
tfidf = dict(zip(words,[tf[w] * math.log2(len(data) / df[w]) for w in words]))
tfidfQuery = [tf[w] * math.log2(len(data) / df[w]) for w in words]

In [32]:
maxTf = [(tf[w],w) for w in words]
maxTf.sort(reverse=True)
maxTfIdf = [(tfidf[w],w) for w in words]
maxTfIdf.sort(reverse=True)

In [41]:
maxTfIdf[:10]

[(7.418581259905156, 'candy'),
 (7.092955525127201, 'jalapeno'),
 (6.856423725257857, 'pull'),
 (6.704253469418681, 'thermometer'),
 (6.7035014782414315, 'sandwich'),
 (6.6997473906667695, 'dredge'),
 (6.68186251235447, 'wide'),
 (6.6518129136030675, 'halfway'),
 (6.493296513199343, 'generously'),
 (6.330029893384327, 'during')]

In [42]:
max(tfidfQuery)

7.418581259905156

In [43]:
def Cosine(x1,x2):
    numer = 0
    norm1 = 0
    norm2 = 0
    for a1,a2 in zip(x1,x2):
        numer += a1*a2
        norm1 += a1**2
        norm2 += a2**2
    if norm1*norm2:
        return numer / math.sqrt(norm1*norm2)
    return 0

In [44]:
similarities = []
for rev2 in data:
    tf = defaultdict(int)
    r = ''.join([c for c in rev2['steps'].lower() if not c in sp])
    for w in r.split():
        # Note = rather than +=
        tf[w] = 1
    tfidf2 = [tf[w] * math.log2(len(data) / df[w]) for w in words]
    similarities.append((Cosine(tfidfQuery, tfidf2), rev2['steps']))

In [45]:
similarities.sort(reverse=True)

In [46]:
similarities[:10]

[(1.0,
  'season chicken breasts with kosher salt\tlet sit at least 5 minutes\tfill a wide , shallow nonreactive bowl or casserole dish with buttermilk\tadd the chicken and soak in the refrigerator for 1 hour up to overnight\tfor the vinaigrette: combine mustard , vinegar and salt in a bowl\tslowly whisk in olive oil until well blended\tfor the coleslaw: macerate onions in the cup of red wine vinegar , and let sit at least 20 minutes\tremove onions and discard vinegar\ttoss onions with jalapeno , parsley , cabbage and salt\ttoss with vinaigrette until evenly coated\tto fry chicken: pour vegetable oil into a large stockpot\tdo not fill up more than halfway , or the oil could splatter\tbring oil up to 365 , using a digital thermometer / candy thermometer to monitor the heat\tprepare the the breading while waiting for oil to heat up\tin a wide shallow bowl , mix flour , cayenne , salt and pepper\tpull a chicken breast out of the buttermilk one by one , letting excess drip off , and dredge

In [116]:
from scipy.sparse import lil_matrix

In [114]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['steps'].lower() if not c in sp])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    feat.append(len(datum['steps']))
    return feat

In [119]:
nf = len(feature(data[0]))

X = lil_matrix((len(data), nf))

for i in range(len(data)):
    #if not (i % 1000):
        #print(i)
    x = feature(data[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]

y = [d['minutes'] for d in data]

In [ ]:
Xtrain = X[:NTrain]
yTrain = y[:NTrain]
Xvalid = X[NTrain:]
Yvalid = y[NTrain:]

mod = linear_model.LinearRegression()

mod.fit(Xtrain, yTrain)

pred = mod.predict(Xvalid)
sum([x**2 for x in (Yvalid - pred)]) / len(Yvalid)

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer

In [51]:
train = ['The sky is blue.','The sun is bright.']
test = ['The sun in the sky is bright', 'We can see the shining sun, the bright sun.']

In [56]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

In [57]:
count_wm = countvectorizer.fit_transform(train)
tfidf_wm = tfidfvectorizer.fit_transform(train)

In [59]:
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()

In [60]:
count_tokens

['blue', 'bright', 'sky', 'sun']

In [61]:
tfidf_tokens

['blue', 'bright', 'sky', 'sun']

In [63]:
df_countvect = pd.DataFrame(data = count_wm.toarray(),index = ['Doc1','Doc2'],columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)

In [64]:
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

Count Vectorizer

      blue  bright  sky  sun
Doc1     1       0    1    0
Doc2     0       1    0    1

TD-IDF Vectorizer

          blue    bright       sky       sun
Doc1  0.707107  0.000000  0.707107  0.000000
Doc2  0.000000  0.707107  0.000000  0.707107


In [65]:
terms = countvectorizer.fit_transform(train)
term_vectors  = countvectorizer.transform(test)
print("Sparse Matrix form of test data : \n")
print(term_vectors.todense())

Sparse Matrix form of test data : 

[[0 1 1 1]
 [0 1 0 2]]


In [66]:
tm = tfidfvectorizer.transform(test)
df_tfidfvect = pd.DataFrame(data = tm.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)


TD-IDF Vectorizer

      blue    bright      sky       sun
Doc1   0.0  0.577350  0.57735  0.577350
Doc2   0.0  0.447214  0.00000  0.894427


In [67]:
countvectorizer = CountVectorizer(analyzer='word' , stop_words='english')
terms = countvectorizer.fit_transform(train)
term_vectors  = countvectorizer.transform(test)
print("Sparse Matrix form of test data : \n")
print(term_vectors.todense())

Sparse Matrix form of test data : 

[[0 1 1 1]
 [0 1 0 2]]


In [70]:
tfidf = TfidfTransformer(norm='l2')
term_vectors.todense()
print(term_vectors.todense())

[[0 1 1 1]
 [0 1 0 2]]


In [71]:
tfidf.fit(term_vectors)
tf_idf_matrix = tfidf.transform(term_vectors)
print("\nVector of idf \n")
print(tfidf.idf_)
print("\nFinal tf-idf vectorizer matrix form :\n")
print(tf_idf_matrix.todense())


Vector of idf 

[2.09861229 1.         1.40546511 1.        ]

Final tf-idf vectorizer matrix form :

[[0.         0.50154891 0.70490949 0.50154891]
 [0.         0.4472136  0.         0.89442719]]


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
train = ('The sky is blue.','The sun is bright.')
test = ('The sun in the sky is bright', 'We can see the shining sun, the bright sun.')
# instantiate the vectorizer object
# use analyzer is word and stop_words is english which are responsible for remove stop words and create word vocabulary
tfidfvectorizer = TfidfVectorizer(analyzer='word' , stop_words='english',)
tfidfvectorizer.fit(train)
tfidf_train = tfidfvectorizer.transform(train)
tfidf_term_vectors  = tfidfvectorizer.transform(test)
print("Sparse Matrix form of test data : \n")
tfidf_term_vectors.todense()

Sparse Matrix form of test data : 



matrix([[0.        , 0.57735027, 0.57735027, 0.57735027],
        [0.        , 0.4472136 , 0.        , 0.89442719]])

In [3]:
data = []

for d in readGz("trainRecipes.json.gz"):
    data.append(d)
    
NTrain = (9*len(data))//10
dataTrain = data[:NTrain]

sp = set(string.punctuation)

Train = []

wordCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['steps'].lower() if not c in sp])
    Train.append(r)
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

NW = 1000

words = [x[1] for x in counts[:NW]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [4]:
Train[:2]

['heat a ridged griddle pan\tlightly brush the tomato slices and bread with some olive oil\tcook the tomato slices first  for at least 5 minutes\twhen they are almost ready  toast the bread in the same pan until well barmarked\tin the meantime  pour a little olive oil into a small frying pan and crack in the egg\tallow it to set for a minute or so and add the garlic and chilli\tcook for a couple of minutes  spooning the hot oil over the egg until cooked to your liking\tplace the griddled bread on a plate and quickly spoon the tomatoes on top\tthrow the chives into the egg pan and splash in the balsamic vinegar\tseason well  then slide the egg on to the tomatoes and drizzle the pan juices on top\tserve immediately  with a good cup of tea ',
 'preheat oven to 350f\tin 13x9inch baking pan  melt butter in oven\tsprinkle crumbs evenly over butter\tpour milk evenly over crumbs\ttop with remaining ingredients\tpress down firmly\tbake 2530 minutes or until lightly browned\tcool completely  chi

In [6]:
tfidfvectorizer = TfidfVectorizer(analyzer='word' , stop_words='english',)
tfidfvectorizer.fit(Train)

TfidfVectorizer(stop_words='english')

In [110]:
tfidf_train = tfidfvectorizer.transform(Train)

In [7]:
efidfv = TfidfVectorizer(analyzer='word',stop_words= 'english')
tfidf_wm = efidfv.fit_transform(Train)

In [8]:
tfidf_wm

<200000x51342 sparse matrix of type '<class 'numpy.float64'>'
	with 8649949 stored elements in Compressed Sparse Row format>

In [11]:
y = [d['minutes'] for d in data]

In [48]:
vwm

<10000x49919 sparse matrix of type '<class 'numpy.float64'>'
	with 428533 stored elements in Compressed Sparse Row format>

In [46]:
trainX = Train[:190000]
trainY = y[:190000]
twm = efidfv.fit_transform(trainX)

valid = Train[190000:]
validY = y[190000:]
vwm = efidfv.transform(valid)

mod = linear_model.Ridge(0.001, fit_intercept=False)
mod.fit(twm, trainY)
predictValid = mod.predict(vwm)
MSEvalid = sum((validY - predictValid)**2)/len(validY)
print("l = " + str(l) + ", validation MSE = " + str(MSEvalid))

l = 0.001, validation MSE = 3511.5993917757532


In [56]:
bestModel = None
bestVal = None
bestLamb = None

trainX = Train
trainY = y
twm = efidfv.fit_transform(trainX)

valid = Train[190000:]
validY = y[190000:]
vwm = efidfv.transform(valid)

lss = [0.1, 1, 10, 100, 1000, 10000]
ls = [1]

for l in ls:
    mod = linear_model.Ridge(l, fit_intercept=False)
    mod.fit(twm, trainY)
    predictValid = mod.predict(vwm)
    MSEvalid = sum((validY - predictValid)**2)/len(validY)
    print("l = " + str(l) + ", validation MSE = " + str(MSEvalid))
    if bestVal == None or MSEvalid < bestVal:
        bestVal = MSEvalid
        bestModel = mod
        bestLamb = l
            
print(bestVal, ' , ', bestLamb)

l = 1, validation MSE = 2664.0598961028663
2664.0598961028663  ,  1


In [53]:
pred = bestModel.predict(vwm)
MSEvalid = sum((validY - predictValid)**2)/len(validY)
print("l = " + str(l) + ", validation MSE = " + str(MSEvalid))

ValueError: dimension mismatch

In [57]:
### Cook-time prediction baseline: Regress based on the length of the instructions

Test=[]

predictions = open("predictions_Minutes.txt", 'w')
predictions.write("recipe_id,prediction\n")
for d in readGz("testRecipes.json.gz"):
    r = ''.join([c for c in d['steps'].lower() if not c in sp])
    Test = [r]
    twm = efidfv.transform(Test)
    pred = bestModel.predict(twm)
    predictions.write(d['recipe_id'] + ',' + str(pred[0]) + '\n')

predictions.close()

In [42]:
twm = efidfv.transform([Train[5]])
pred = model.predict(twm)
pred

array([73.60770004])

In [43]:
y[5]

75

In [21]:
Train[:2]

['heat a ridged griddle pan\tlightly brush the tomato slices and bread with some olive oil\tcook the tomato slices first  for at least 5 minutes\twhen they are almost ready  toast the bread in the same pan until well barmarked\tin the meantime  pour a little olive oil into a small frying pan and crack in the egg\tallow it to set for a minute or so and add the garlic and chilli\tcook for a couple of minutes  spooning the hot oil over the egg until cooked to your liking\tplace the griddled bread on a plate and quickly spoon the tomatoes on top\tthrow the chives into the egg pan and splash in the balsamic vinegar\tseason well  then slide the egg on to the tomatoes and drizzle the pan juices on top\tserve immediately  with a good cup of tea ',
 'preheat oven to 350f\tin 13x9inch baking pan  melt butter in oven\tsprinkle crumbs evenly over butter\tpour milk evenly over crumbs\ttop with remaining ingredients\tpress down firmly\tbake 2530 minutes or until lightly browned\tcool completely  chi

In [112]:
tfidf_train.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [12]:
y[:10]

[10, 40, 50, 25, 55, 75, 40, 20, 55, 105]

In [98]:
tfidf_train.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [99]:
tfidf_tokens = tfidfvectorizer.get_feature_names()

In [106]:
tfidf_train[0, 9]

0.0

In [104]:
tfidf_tokens[454]

'blanch'

In [91]:
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')

In [92]:
terms = countvectorizer.fit_transform(Train)
term_vectors  = countvectorizer.transform(Train)

In [113]:
# print("Sparse Matrix form of test data : \n")
# print(term_vectors.todense())